# This notebook helps develop the goesscraper script

In [24]:
import nesdis_aws
import nesdis_gml_synergy.satlab as ngs


In [15]:
# def process_generic(
product = 'AOD'
stations = {'abb': 'TBL', 'name': 'Table Mountain (CO)', 'lat': 40.12498, 'lon': -105.2368}
satellite = 17
path2log = None
start='2019-01-01 19:00:00'
end='2019-01-01 19:05:00'
# path2processed =  '/nfs/stu3data2/Satellite_data/goes/{satellite}/ABI_L2_{product}_projected2surfrad/'
path2processed ='/export/htelg/tmp/'
#path2folder_local=f'/nfs/stu3data2/Satellite_data/goes/{satellite}/ABI_L2_{product}/',
path2folder_local = '/export/htelg/tmp/'
file_prefix = 'ABI_L2_{product}_projected2surfrad'
no_of_cpu = 1,
reverse = False,
verbose = False,
test = False
         

# step by step for testing

## download the file without processing

In [16]:
query = nesdis_aws.nesdis_aws.AwsQuery(
                                        path2folder_local=path2folder_local,
                                        satellite=satellite,
                                        product=f'ABI-L2-{product}',
                                        scan_sector='C',
                                        start=start,
                                        end=end,
                                        process = dict(#concatenate = 'daily',
                                                       function = lambda row: ngs.projection_function_multi(row), #,, alt_var),
                                                       prefix = file_prefix.format(product = product),
                                                       path2processed = path2processed,
                                                        )
                                        # keep_files=True
                                      )

In [18]:
query.download()

UnboundLocalError: local variable 'out' referenced before assignment

In [23]:
query.workplan.iloc[0]

path2file_aws                noaa-goes17/ABI-L2-AODC/2019/001/19/OR_ABI-L2-...
path2file_local              /export/htelg/tmp/OR_ABI-L2-AODC-M3_G17_s20190...
path2file_local_processed    /export/htelg/tmp/ABI_L2_AOD_projected2surfrad...
Name: 2019-01-01 19:02:19, dtype: object

In [38]:
reload(ngs)

<module 'nesdis_gml_synergy.satlab' from '/export/htelg/prog/nesdis_gml_synergy/nesdis_gml_synergy/satlab.py'>

In [39]:
ds = ngs.projection_function(query.workplan.iloc[0], stations=stations, test=True )
ds

In [40]:
projection = ds

In [41]:
projection.projection2area

/export/htelg/programs/anaconda3/envs/py310telg/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/export/htelg/programs/anaconda3/envs/py310telg/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/export/htelg/programs/anaconda3/envs/py310telg/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/export/htelg/programs/anaconda3/envs/py310telg/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/export/htelg/programs/anaconda3/envs/py310telg/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1878: Run

<xarray.Dataset>
Dimensions:              (radius: 5, stats: 3, data_quality: 5, site: 1)
Coordinates:
  * radius               (radius) int64 5 10 25 50 100
  * stats                (stats) object 'median' 'mean' 'std'
    t                    datetime64[ns] 2019-01-01T19:03:37.685171968
  * site                 (site) <U3 'TBL'
  * data_quality         (data_quality) <U15 'high' 'high_medium' ... 'low'
Data variables:
    AOD                  (data_quality, radius, stats, site) float32 nan ... nan
    num_of_valid_points  (data_quality, radius, site) int64 10 39 ... 938 3766

# this is what the script does

In [ ]:
           # ):
path2processed =  path2processed.format(product = product, satellite = satellite)
assert(pl.Path(path2processed).is_dir()), f"Output path does not exist ... generate it!:\n pl.Path('{path2processed}').mkdir()"
query = nesdis_aws.nesdis_aws.AwsQuery(
                                        path2folder_local=path2folder_local,
                                        satellite=satellite,
                                        product=f'ABI-L2-{product}',
                                        scan_sector='C',
                                        start=start,
                                        end=end,
                                        process = dict(#concatenate = 'daily',
                                                       function = lambda row: ngs.projection_function_multi(row), #,, alt_var),
                                                       prefix = file_prefix.format(product = product),
                                                       path2processed = path2processed,
                                                        )
                                        # keep_files=True
                                      )
print('========', flush = True)
print('workplan', flush = True)
print('========', flush = True)
print(f'workplan.shape: {query.workplan.shape}', flush = True)
if reverse:
    query.workplan = query.workplan[::-1]
if test:
    print(query.workplan, flush = True)
    return query.workplan


query.process_parallel(no_of_cpu = no_of_cpu, 
                       process_function=ngs.projection_function_multi, args={'stations': stations}, 
                       path2log = path2log, 
                       subprocess = product,
                       server = socket.gethostname(),
                       comment = '')
print('done', flush = True)
# return